# Create New Customers

In [367]:
from datagen_customer import Customer, Headers
from demographics import make_cities, make_age_gender_dict
from faker import Faker
import pandas as pd
import numpy as np
import datetime
import random
import google.auth
from google.cloud import storage

import pandas_gbq
import os

In [689]:
#service account needs write access into bq project
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/leighaj/Service Accounts/looker-private-demo-d4bcf655c1aa.json'

credentials = credentials, project = google.auth.default(
    scopes=[
        "https://www.googleapis.com/auth/bigquery"
    ]
)

In [2]:
#set random seed for faker
fake = Faker()
Faker.seed(44)

In [3]:
#create 100,000 customers which are saved to file
for _ in range(100000):
    Customer(fake = fake, filename = 'customers.csv')

Fill in Missing Data Pieces

In [14]:
#assign a traffic source
def create_traffic_source(profile):
    traffic_sources = ['Mailer','Email','Call Center','Website','Third Party Advertisement']
    if 'young_adults' in profile:
        #more likely to be third party adverisement or website
        traffic_sources = traffic_sources + [traffic_sources[4],traffic_sources[4],traffic_sources[4
                                    ],traffic_sources[3],traffic_sources[3]]
    elif 'urban' in profile:
        traffic_sources = traffic_sources + [traffic_sources[1],traffic_sources[2],traffic_sources[2
                        ],traffic_sources[3],traffic_sources[3]]
    else:
        traffic_sources = traffic_sources + [traffic_sources[1],traffic_sources[1],traffic_sources[1
                ],traffic_sources[2],traffic_sources[3]]
    
    return np.random.choice(traffic_sources)

In [423]:
#read in csv as a dataframe
clients = pd.read_csv('customers.csv','|',names=['ssn', 'cc_num', 'first', 'last', 'gender', 'street', \
                    'city', 'state', 'zip', 'lat', 'long', 'city_pop', \
                    'job', 'dob', 'acct_num', 'profile'])

In [424]:
clients.head()

,ssn,cc_num,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,acct_num,profile
0,419-67-8880,213115492181911,Cassandra,Perez,F,01284 Gray Knolls,Oswego,NY,13126,43.4394,-76.4613,37205,Retired,1931-12-20,206976967943,adults_50up_female_urban.json
1,134-34-4063,4203099884593616,Amanda,Yu,F,557 Lawrence Mountains Apt. 759,Oak City,NC,27857,35.9597,-77.3004,1298,Sales promotion account executive,1993-09-03,708379739941,adults_2550_female_rural.json
2,753-01-4812,3595871474664876,Denise,Smith,F,304 Tiffany Walks,Turner,ME,4282,44.2557,-70.2494,5734,Retired,1952-02-22,301009463851,adults_50up_female_urban.json
3,590-99-2188,4958155720167,Nichole,Gordon,F,38718 Robert Crossing,Huntington Park,CA,90255,33.9769,-118.2161,75066,Theme park manager,1991-06-24,557000271328,adults_2550_female_urban.json
4,460-29-3616,6546242915895946,Michael,Fitzgerald,M,83922 Alexis Coves,Long Beach,CA,90822,33.7927,-118.1638,471952,Financial risk analyst,1994-04-27,984224770197,adults_2550_male_urban.json


In [425]:
## already exists
#ssn
#first_name
#last_name
#gender
#street
#address
#job
#profile
#dob 

## need to create
#age - create in database
#district_id - keep as null
#client_id - keep as null

#client_id add row number 
clients = clients.reset_index()
clients['disp_id'] = clients['index'].apply(lambda x: x + 13661)
clients['card_id'] = clients['index'].apply(lambda x: x + 1462)

#traffic_source
clients['traffic_source'] = clients['profile'].apply(lambda x: create_traffic_source(x))


Figure out the cards metadata for each user

In [426]:
clients = clients.sample(frac=1)
clients = clients.drop(['index'], axis=1)
clients = clients.reset_index()

clients['card_type'] = None
clients['card_start'] = None
clients['card_end'] = None

Young Adults

In [427]:
#Give 70% of young adults junior card, 20% junior2 card and 6% classic, 3% classic2 and rest gold
length = len(clients)

clients.loc[clients['profile'].str.contains('young'),'card_type'] = clients.loc[
    clients['profile'].str.contains('young')].apply(lambda x: 'Junior' if int(x['index']) < .7*length 
            else 'Junior2' if int(x['index']) <.9*length 
            else 'Classic' if int(x['index']) <.96*length
            else 'Classic2' if int(x['index']) <.99*length else 'Gold', axis=1)

25-50

In [428]:
#Give 60% of classic, 30% classic2, 10% gold 
#length = len(clients.loc[clients['profile'].str.contains('2550')])

clients.loc[clients['profile'].str.contains('2550'),'card_type'] = clients.loc[
    clients['profile'].str.contains('2550')].apply(lambda x: 'Classic' if int(x['index']) < .6*length 
            else 'Classic2' if int(x['index']) <.9*length 
            else 'Gold', axis=1)

50+

In [429]:
#Give 50% of classic, 35% classic2, 15% gold 
#length = len(clients.loc[clients['profile'].str.contains('50up')])

clients.loc[clients['profile'].str.contains('50up'),'card_type'] = clients.loc[
    clients['profile'].str.contains('50up')].apply(lambda x: 'Classic' if int(x['index']) < .5*length 
            else 'Classic2' if int(x['index']) <.35*length
            else 'Gold', axis=1)

In [430]:
# randomly chose a start date that is after november 2015 more than 15 years after their birthday, and before end of this year

clients['card_start'] = clients['dob'].apply(lambda x: 
        datetime.timedelta(days=random.randint(max(15*365,(pd.to_datetime('2015-11-01')-pd.to_datetime(
                        x)).days), (pd.to_datetime(
        '2020-12-31')-pd.to_datetime(x)).days)) + pd.to_datetime(x))

Assign subsequent cards and end dates

In [431]:
# for junior cards - take 80% of cards and create an end date and a new card add date
if 'index' in clients.columns:
    clients = clients.drop(['index'], axis=1)
results = clients.loc[clients['card_type'] == 'Junior'].sample(frac=.9).reset_index()


#randomly generate a normal distribution of time they convert to next card, and the time they take to cancel previous
days_to_add = list(np.random.normal(365, 150, len(results)))
days_to_cancel = list(np.random.normal(30*3, 25, len(results)))
new_card_id = clients['card_id'].max()

new_cards = []

for i in range(len(results)):
    card_id = results.loc[i,'card_id']
    start_date = clients.loc[clients['card_id'] == card_id,'card_start']
    new_card_start_date = pd.to_datetime(start_date.values[0]) + datetime.timedelta(days=days_to_add.pop())
    cancel_date = new_card_start_date + datetime.timedelta(days=days_to_cancel.pop())
    
    
    #update the end date
    clients.loc[clients['card_id'] == card_id,'card_end'] = cancel_date
    
    
    #15% just cancel
    if i < .85*len(results) and new_card_start_date <= pd.to_datetime('2020-12-31'):
        #55% go to Junior2
        #30% go to Classic
        if i<.55*len(results):
            new_card_type = 'Junior2'
        else:
            new_card_type = 'Classic'
        
        new_card_id += 1 
        new_card = clients.loc[clients['card_id'] == card_id].to_dict('records')[0]
        new_card['card_id'] = new_card_id
        new_card['card_type'] = new_card_type
        new_card['card_start'] = new_card_start_date
        new_card['card_end'] = np.nan
        new_card['cc_num'] = fake.credit_card_number()
        new_cards.append(new_card)
    
clients = clients.append(pd.DataFrame(new_cards)) 

In [432]:
clients = clients.loc[clients['card_start']<pd.to_datetime('2020-12-31')]

In [433]:
#for each gold member
#change the original card_type 
#then randomly chose a date between now and when the original card was created to sign up for gold
#also if index is in randomly chosen 10% then chose an end date for the gold card also

results = clients.loc[clients['card_type'] == 'Gold'].sample(frac=1).reset_index()

ix_cl = round(len(results)*.6)
ix_j2 = ix_cl+round(len(results)*.1)
ix_cl2 = ix_j2+round(len(results)*.1)
rand_10 = random.sample(range(0, ix_cl2), round(ix_cl2*.25))


new_cards = []
new_card_id = max(clients['card_id'])
curr_ix = 0

days_to_add = list(np.random.normal(600, 150, len(results)))
days_to_cancel = list(np.random.normal(30*3, 25, len(results)))

for ix in [ix_cl,ix_j2,ix_cl2]:
    for i in range(curr_ix,ix):
        card_id = results.loc[i,'card_id']
        disp_id = clients.loc[clients['card_id'] == card_id,'disp_id'].values[0]
        if ix == ix_cl:
            card_type = 'Classic2'
        elif ix == ix_j2:
            card_type = 'Junior2'
        else:
            card_type = 'Classic'
        
        clients.loc[clients['card_id'] == card_id,'card_type'] = card_type
        
        
        created_date = pd.to_datetime(clients.loc[clients['card_id'] == card_id,'card_start'
                             ].values[0])
        new_card_date = created_date + datetime.timedelta(days=days_to_add.pop()) 
        new_cancel_date = np.nan
        
        if i in rand_10[0:15]:
            new_cancel_date = new_card_date + datetime.timedelta(days=days_to_cancel.pop()) 
        
        elif i in rand_10[15:]:
            if created_date != datetime.datetime(2020, 12, 31, 0, 0):
                clients.loc[clients['card_id'] == card_id,'card_end'] = created_date + datetime.timedelta(days=random.randrange((
                    datetime.datetime(2020, 12, 31, 0, 0) - created_date).days)) 

        
        #update cards
        new_card_id += 1 
        new_card = clients.loc[clients['card_id'] == card_id].to_dict('records')[0]
        new_card['card_id'] = new_card_id
        new_card['card_type'] = 'Gold'
        new_card['card_start'] = new_card_date
        new_card['card_end'] = new_cancel_date
        new_card['cc_num'] = fake.credit_card_number()
        new_cards.append(new_card)
        
    curr_ix = ix
clients = clients.append(pd.DataFrame(new_cards)) 

In [434]:
#add clients staging
pd.DataFrame(clients).to_gbq('finserv_staging.new_clients',if_exists = 'fail')



0it [00:00, ?it/s]

1it [00:26, 26.64s/it]


# Create New Transactions
### If you already have a customers.csv file then you only need to run these cells, but you will need to import the clients dataframe from BQ to get card_id (as shown in comment below)

In [690]:
#clients = pd.read_gbq('select * from finserv_staging.new_clients')



Downloading:   0%|          | 0/127038 [00:00<?, ?rows/s]

Downloading:  31%|███       | 39559/127038 [00:20<00:45, 1939.37rows/s]

Downloading:  62%|██████▏   | 78577/127038 [00:32<00:21, 2206.90rows/s]

Downloading:  93%|█████████▎| 118074/127038 [00:39<00:03, 2667.72rows/s]

Downloading: 100%|██████████| 127038/127038 [00:42<00:00, 2996.71rows/s]


In [435]:
from os import listdir
from os.path import isfile, join
from datetime import date
import faker
import profile_weights
from datagen_transaction import T_Customer, create_header
import os
import multiprocessing as mp
#print("Number of processors: ", mp.cpu_count())
from google.cloud import storage

In [436]:
profile_files = [f for f in listdir('profiles/'
                                   ) if isfile(join('profiles/', f))]
profile_files = [join('profiles/', f) 
                 for f in profile_files if f.split('_')[0] in ('adults','young')]

In [437]:
customers_file = open('customers.csv', 'r').readlines() 

In [438]:
headers = ''
for h in ['ssn', 'cc_num', 'first', 'last', 'gender', 'street', \
            'city', 'state', 'zip', 'lat', 'long', 'city_pop', \
            'job', 'dob', 'acct_num', 'profile']:
    headers += h + '|'
headers = headers[:-1]
headers = create_header(headers)

ssn|cc_num|first|last|gender|street|city|state|zip|lat|long|city_pop|job|dob|acct_num|profile|trans_num|trans_date|trans_time|unix_time|category|amt|is_fraud|merchant|merch_lat|merch_long


In [ ]:
client = storage.Client()
bucket = client.get_bucket('retail_banking')

In [511]:
def base(m):
    counter = 0
    txns = []
    cust_counter = 0
    
    
    n=(m+1)*10000
    print((m*10000)+1000,n)
    for i, customer_card in clients[(m*10000)+2500:n].iterrows():
        for m in profile_files:
            pro_name = m.split('profiles')[-1]
            pro_name = pro_name[1:]
            parse_index = m.index('profiles') + 9
            m_fraud = m[:parse_index] +'fraud_' + m[parse_index:]
            pro = open(m, 'r').read()
            pro_fraud = open(m_fraud, 'r').read()
            pro_name_fraud = 'fraud_' + pro_name

            curr_profile = pro_name
            curr_fraud_profile = pro_name_fraud
            profile_name = m

            card_id = customer_card['card_id']
            create_d = pd.to_datetime(customer_card['card_start'])

            end_d = pd.to_datetime(customer_card['card_end'])
            if end_d is None:
                end_d = pd.to_datetime(date(2021,1,1))
                if (end_d-create_d).days < 10:
                    end_d = pd.to_datetime(date(2021,1,10))

            profile = profile_weights.Profile(pro,create_d,end_d)
            cust = customers_file[[i for i, s in enumerate(customers_file) if customer_card['ssn'] in s][0]]
            t_cust = T_Customer(cust, profile, headers, fake)

            if t_cust.attrs['profile'] == curr_profile:
                is_fraud= 0
                fraud_flag = random.randint(0,100) 
                fraud_dates = []

                merch = pd.read_csv('demographic_data/merchants.csv' , sep='|')
                # decide if we generate fraud or not
                if fraud_flag < 30: #11->25
                    fraud_interval = random.randint(1,1) #7->1
                    inter_val = (end_d-create_d).days-7
                    # rand_interval is the random no of days to be added to start date
                    if inter_val <= 0:
                        inter_val = 1
                    rand_interval = random.randint(1, inter_val)
                    #random start date is selected
                    newstart = create_d + datetime.timedelta(days=rand_interval)
                    # based on the fraud interval , random enddate is selected
                    newend = newstart + datetime.timedelta(days=fraud_interval)
                    # we assume that the fraud window can be between 1 to 7 days #7->1
                    profile = profile_weights.Profile(pro_fraud, newstart, newend)
                    t_cust = T_Customer(cust, profile, headers, fake)
                    is_fraud = 1
                    temp_tx_data = profile.sample_from(is_fraud)
                    fraud_dates = temp_tx_data[3]
                    txns = t_cust.print_trans(temp_tx_data,is_fraud, fraud_dates,merch)
                    #parse_index = m.index('profiles/') + 9
                    #m = m[:parse_index] +'fraud_' + m[parse_index:]

                # we're done with fraud (or didn't do it) but still need regular transactions
                # we pass through our previously selected fraud dates (if any) to filter them
                # out of regular transactions
                profile = profile_weights.Profile(pro, create_d, end_d)
                is_fraud =0
                temp_tx_data = profile.sample_from(is_fraud)
                if txns is not None:
                    txns = txns + t_cust.print_trans(temp_tx_data, is_fraud, fraud_dates, merch)
                else:
                    txns = t_cust.print_trans(temp_tx_data, is_fraud, fraud_dates, merch)


                txns = pd.DataFrame(txns)
                #add in customer id
                txns['card_id'] = customer_card['card_id']

                #if the file isnt more than 5GB then append the transactions to it otherwise create new file
                if os.path.getsize(filename)/1000000000 > 5:
                    #upload the file to gcp storage bucket
                    bucket.blob(filename).upload_from_filename(filename, 'text/csv')
                    counter += 1
                filename = 'trans_thread_'+m+counter+'.csv'
                txns.to_csv(filename, mode='a', header=False, sep='|')
                
                loop_counter += 1
                
                
                print(m,'Thread, added ', len(txns),' to the table, for card ', card_id, ' card #: ', 
                      loop_counter, ' in file: ', filename)
                txns = None
    return ('done ', n)

In [514]:
from multiprocessing import Process

procs = []
proc = Process(target=base) 
procs.append(proc)
proc.start()

# instantiating process with arguments
for m in range(12):
    print(m)
    proc = Process(target=base, args=(m,))
    procs.append(proc)
    proc.start()

# complete the processes
for proc in procs:
    proc.join()
    

0
1
2
3
4
5
6
7
41000 50000
11000 20000
31000 40000
8
21000 30000
11000 20000
9
1000 10000
51000 60000
61000 70000
10
11
71000 80000
81000 90000
91000 100000
101000 110000
111000 120000


Process Process-196:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-511-e244426be568>", line 38, in base
    profile = profile_weights.Profile(pro,create_d,end_d)
  File "/Users/leighaj/data_gen/retail_banking_dataset_gen/profile_weights.py", line 19, in __init__
    self.make_weights()
  File "/Users/leighaj/data_gen/retail_banking_dataset_gen/profile_weights.py", line 173, in make_weights
    self.date_weights()
  File "/Users/leighaj/data_gen/retail_banking_dataset_gen/profile_weights.py", line 160, in date_weights
    self.prep_years()
  File "/Users/leighaj/data_gen/retail_banking_dataset_gen/profile_weights.py", line 115, in prep_years
    years = [y for y in range(self.start.year, self.end.year+1)]
TypeError: 'float' object cannot be inter

30000 , added  1072  to the table, for card  35474 Number  1
80000 , added  1922  to the table, for card  55279 Number  1
90000 , added  3090  to the table, for card  5394 Number  1
40000 , added  1593  to the table, for card  61299 Number  1
50000 , added  758  to the table, for card  53510 Number  1
60000 , added  2762  to the table, for card  41126 Number  1
70000 , added  1514  to the table, for card  16282 Number  1
100000 , added  5207  to the table, for card  55626 Number  1
10000 , added  2494  to the table, for card  93753 Number  1
20000 , added  2086  to the table, for card  82427 Number  1
20000 , added  2086  to the table, for card  82427 Number  1
10000 , added  1605  to the table, for card  36351 Number  2
80000 , added  1322  to the table, for card  57773 Number  2
50000 , added  6524  to the table, for card  95677 Number  2
40000 , added  127  to the table, for card  9242 Number  2
100000 , added  1755  to the table, for card  17352 Number  2
20000 , added  45  to the 

100000 , added  3132  to the table, for card  71142 Number  12
10000 , added  1692  to the table, for card  90090 Number  13
20000 , added  3093  to the table, for card  11425 Number  14
60000 , added  8020  to the table, for card  59386 Number  12
30000 , added  2654  to the table, for card  29739 Number  14
100000 , added  1686  to the table, for card  4002 Number  13
90000 , added  6172  to the table, for card  12896 Number  13
50000 , added  1401  to the table, for card  81603 Number  13
40000 , added  5514  to the table, for card  14510 Number  14
20000 , added  244  to the table, for card  84090 Number  15
70000 , added  5550  to the table, for card  19050 Number  13
80000 , added  1224  to the table, for card  35568 Number  13
10000 , added  242  to the table, for card  89836 Number  14
20000 , added  305  to the table, for card  84090 Number  15
90000 , added  881  to the table, for card  86807 Number  14
70000 , added  0  to the table, for card  6082 Number  14
60000 , added  

90000 , added  1653  to the table, for card  61745 Number  26
10000 , added  3834  to the table, for card  101157 Number  25
100000 , added  4768  to the table, for card  98286 Number  25
80000 , added  3214  to the table, for card  66156 Number  25
50000 , added  4380  to the table, for card  71650 Number  24
20000 , added  2758  to the table, for card  29140 Number  26
30000 , added  4248  to the table, for card  91422 Number  26
60000 , added  1387  to the table, for card  89650 Number  24
40000 , added  2082  to the table, for card  84995 Number  26
100000 , added  3760  to the table, for card  3232 Number  26
20000 , added  856  to the table, for card  22854 Number  27
80000 , added  22  to the table, for card  33454 Number  26
70000 , added  3440  to the table, for card  68961 Number  26
100000 , added  426  to the table, for card  46274 Number  27
30000 , added  1144  to the table, for card  32806 Number  27
90000 , added  4649  to the table, for card  17704 Number  27
50000 , a

...

20000 , added  10560  to the table, for card  27018 Number  1378
20000 , added  4098  to the table, for card  85000 Number  1379
10000 , added  0  to the table, for card  26248 Number  1383
80000 , added  341  to the table, for card  62923 Number  1382
50000 , added  771  to the table, for card  91954 Number  1375
40000 , added  4683  to the table, for card  41955 Number  1384
70000 , added  0  to the table, for card  92399 Number  1366
100000 , added  0  to the table, for card  42183 Number  1390
80000 , added  482  to the table, for card  84245 Number  1383
20000 , added  530  to the table, for card  9575 Number  1380
90000 , added  3228  to the table, for card  40200 Number  1378
60000 , added  4599  to the table, for card  55287 Number  1383
30000 , added  981  to the table, for card  92843 Number  1389
50000 , added  946  to the table, for card  89871 Number  1376
40000 , added  381  to the table, for card  55018 Number  1385
20000 , added  1437  to the table, for card  60107 Numb

Process Process-191:
Process Process-184:
Process Process-190:
Process Process-185:
Process Process-186:
Process Process-188:
Process Process-187:
Process Process-189:
Process Process-193:
Process Process-192:
Process Process-194:
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pr

KeyboardInterrupt
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 4405, in get_value
    return self._engine.get_value(s, k, tz=getattr(series.dtype, "tz", None))
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py", line 871, in __getitem__
    result = self.index.get_value(self, key)
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 4405, in get_value
    return self._engine.get_value(s, k, tz=getattr(series.dtype, "tz", None))
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 4379, in get_value
    s = extract_array(series, extract_numpy=True)
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py", line 871, in __getitem__
    result = self.index.get_value(self, key)
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 4379, in get_value
    s = extract_array(series, extract_numpy=True)
  File "/opt/ana

In [646]:
# with open("transactions.csv") as myfile:
#     file = myfile.readlines()
    
# file = [ x.replace('"','').replace('|||||||||||','').replace('"|"',',').replace('Zieme, Bode and Dooley','Bode and Dooley') for x in file]

# new_txns = pd.DataFrame([sub.split("|") if i > 407122 else sub.split(",") for i,sub in enumerate(file)])

In [662]:
# new_txns2 = new_txns[new_txns[13].isnull()]
# to_fix = new_txns[~new_txns[13].isnull()]

# to_fix[9] = to_fix[9] + to_fix[10] 
# to_fix[10] = to_fix[11]
# to_fix[11] = to_fix[12]
# to_fix[12] = to_fix[13]

# new_txns2 = new_txns2.loc[:,[1,2,3,4,5,6,7,8,9,10,11,12]]
# to_fix = to_fix.loc[:,[1,2,3,4,5,6,7,8,9,10,11,12]]

# new_txns3 = new_txns2.append(to_fix)
# new_txns3 = new_txns3.reset_index(drop=True)

# new_txns3.columns = ['cc_num','trans_id','trans_date','trans_time','epoch_time','category','amount','is_fraud',
#                     'merchant','merchant_lat','merchant_lon','card_id']
# new_txns3['card_id'] = new_txns3['card_id'].replace('\n','')